<a href="https://colab.research.google.com/github/alelorys/chatbot-flask/blob/main/chatBot_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok
!pip install pystempel
!pip install transformers
!pip install tflearn
!pip install pyspellchecker 

     |████████████████████████████████| 2.7 MB 4.0 MB/s 
     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 20.7 MB/s 
     |████████████████████████████████| 636 kB 54.6 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 107 kB 4.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=9e1dc3c6dd4578416bd9f42686612431ddce35f1b2b1a7beb73532df9d2c583b
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
     |████████████████████████████████| 2.7 MB 4.2 MB/s 


In [35]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json
import pickle
import random
import difflib

import nltk
import numpy
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml
from stempel import StempelStemmer
nltk.download('punkt')

app = Flask(__name__)
run_with_ngrok(app)

stemmer = StempelStemmer.polimorf()

with open("/content/drive/MyDrive/Colab Notebooks/chatbot/q&a.json") as file:
    data = json.load(file)

try:
    with open("/content/drive/MyDrive/Colab Notebooks/chatbot/chatbot.pickle", "rb") as file:
        words, labels, training, output = pickle.load(file)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["questionAnswer"]:
        for pattern in intent["patterns"]:
            wrds = pattern.split(" ")
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("/content/drive/MyDrive/Colab Notebooks/chatbot/chatbot.pickle", "wb") as file:
        pickle.dump((words, labels, training, output), file)

try:
    yaml_file = open('/content/drive/MyDrive/Colab Notebooks/chatbot/chatbotmodel.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    myChatModel = model_from_yaml(loaded_model_yaml)
    myChatModel.load_weights("/content/drive/MyDrive/Colab Notebooks/chatbot/chatbotmodel.h5")
    print("Loaded model from disk")

except:
    # Make our neural network
    myChatModel = Sequential()
    myChatModel.add(Dense(8, input_shape=[len(words)], activation='relu'))
    myChatModel.add(Dense(len(labels), activation='softmax'))

    # optimize the model
    myChatModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # train the model
    myChatModel.fit(training, output, epochs=1000, batch_size=8)

    # serialize model to yaml and save it to disk
    model_yaml = myChatModel.to_yaml()
    with open("/content/drive/MyDrive/Colab Notebooks/chatbot/chatbotmodel.yaml", "w") as y_file:
        y_file.write(model_yaml)

    # serialize weights to HDF5
    myChatModel.save_weights("/content/drive/MyDrive/Colab Notebooks/chatbot/chatbotmodel.h5")
    print("Saved model from disk")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = s.split(" ")

    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
          seq = difflib.SequenceMatcher(None,w,se)
          d = seq.ratio()*100
          if w == se:
            bag[i] = 1
          if d >= 75:
            bag[i] = 1
          
         
            
    return numpy.array(bag)

def chatWithBot(userMess):
    currentText = bag_of_words(userMess, words)
    currentTextArray = [currentText]
    numpyCurrentText = numpy.array(currentTextArray)
    if numpy.all((numpyCurrentText == 0)):
      answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
      return answer

    result = myChatModel.predict(numpyCurrentText[0:1])
    result_index = numpy.argmax(result)
    tag = labels[result_index]
    if result[0][result_index] > 0.6:
        for tg in data["questionAnswer"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        answer = random.choice(responses)
        return answer

    else:
        answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
        return answer

def chat(userMess):
    

    while True:
        inp = userMess
        if inp.lower() == "quit":
            break
        if inp.lower() == "" or inp.lower() == " ":
          answer = "Musisz coś napisać!"
          return answer
        
        return chatWithBot(inp)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userMess = request.args.get("msg")
    return chat(userMess)


if __name__ == "__main__":
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading:  99%|█████████▉| 11302016/11368252 [00:14<00:00, 455785.47bytes/s]

Loaded model from disk
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://303eb8ddbb0c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Aug/2021 07:46:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:46:07] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [17/Aug/2021 07:46:09] "GET /favicon.ico HTTP/1.1" 404 -
Loading: 100%|██████████| 11368252/11368252 [00:26<00:00, 455785.47bytes/s]127.0.0.1 - - [17/Aug/2021 07:46:21] "GET /get?msg=dzien%20dobry HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:46:26] "GET /get?msg= HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:46:39] "GET /get?msg=Kiedy%20zaczynaja%20się%20zajęcia HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:46:55] "GET /get?msg=Jak%20można%20dojechać%20do%20akademii%3F HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:47:07] "GET /get?msg=Jak%20wyglada%20nauka HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 07:47:13] "GET /get?msg=Dziękuje HTTP/1.1" 200 -


In [ ]:
def chat(userMess):
    
    while True:
        inp = userMess
        if inp.lower() == "quit":
            break
        if inp.lower() == "":
          responses = "Musisz coś napisać"
          #print("Czesio: Musisz coś napisać!")
        else:
          results = model.predict([bag_of_words(inp, words)])
          results_index = numpy.argmax(results)
          tag = labels[results_index]

          for tg in data["questionAnswer"]:
              if tg['tag'] == tag:
                  responses = tg['responses']
              

          answer = random.choice(responses)

        return answer



8